In [1]:
import os
import datetime
import numpy as np
import matplotlib.pyplot as plt 

from pdf_utils import get_R_z_string

"""
    Obtain the stacked-nonzero-PDF-Fisher matrix
    for different redshifts.
    - not all redshifts have all smoothing scales (missing 35 Mpc on some)
""";
delfi_data_dir = "/Users/Jed.Homer/phd/lfi/jaxdelfi/data/"
cmap = "bwr"
dpi = 100

ModuleNotFoundError: No module named 'pdf_utils'

In [ ]:
resolution = 1024
all_redshifts = [
    0.0, 0.5, 1.0, 2.0, 3.0]
all_R_values = [
    "5.0", "10.0", "15.0", "20.0", "25.0", "30.0", "35.0"]

redshifts = all_redshifts[:1]
R_values = all_R_values[1:4]

n_radii = len(R_values)
redshifts, R_values

In [2]:
# Density cut values
cora = True 
if cora:
    p_value_min = 0.03
    p_value_max = 0.90
else:
    p_value_min = 0.001
    p_value_max = 0.999

In [ ]:
# Indices of R values in all R values list
R_idx = [all_R_values.index(R) for R in R_values]
z_idx = [all_redshifts.index(z) for z in redshifts]
print(f"Running for R values:\n\t{[all_R_values[R_idx_] for R_idx_ in R_idx]}.")
print(f"Running for z values:\n\t{[all_redshifts[z_idx_] for z_idx_ in z_idx]}.")
R_idx, z_idx

In [ ]:
Rz_string = get_R_z_string(R_values, redshifts)
Rz_string = "cora_" + Rz_string if cora else Rz_string
Rz_string

In [ ]:
fiducial_pdfs_dir = os.path.join(
    "/Users/Jed.Homer/phd/lfi/quijote_lh_pdf_data/all_fiducial_pdfs/")

all_fiducial_pdfs = np.load(
    #os.path.join(fiducial_pdfs_dir, f"FIDUCIAL_PDFS_{resolution}.npy"))
    #f"/Users/Jed.Homer/phd/lfi/jaxdelfi_pdf/data/ALL_FIDUCIAL_PDFS_{resolution}_egg.npy")
    f"/Users/Jed.Homer/phd/lfi/jaxdelfi/data/ALL_FIDUCIAL_PDFS_REFRESHED_{resolution}.npy")

# Common to all pdfs (values of delta in the middle of the bins?)
deltas = np.load("/Users/Jed.Homer/phd/lfi/quijote_lh_pdf_data/DELTAS.npy")

delta_bin_edges = np.geomspace(1e-2, 1e2, num=100) 
D_deltas = delta_bin_edges[1:] - delta_bin_edges[:-1]

In [ ]:
#derivatives_dir = "/Users/Jed.Homer/phd/lfi/quijote_lh_pdf_data/derivatives/"
derivatives_dir = "/Users/Jed.Homer/phd/lfi/quijote_lh_pdf_data/DERIVATIVES_REFRESHED/"

#### cut 

In [3]:
# Hartlap factor, should this or its inverse multiply C^-1?
n_z, n_fiducial_pdfs, n_pdf_dims, n_R = all_fiducial_pdfs.shape #fiducial_pdfs_stacked.shape
n_bins_pdf = n_pdf_dims
data_dim = n_z * n_pdf_dims * n_R
H = (n_fiducial_pdfs - 2. - data_dim) / (n_fiducial_pdfs - 1.)
H

NameError: name 'all_fiducial_pdfs' is not defined

In [ ]:
# Select and flatten fiducial pdfs for chosen z, R
all_fiducial_pdfs_ = all_fiducial_pdfs[:, :, :n_bins_pdf, :]

print(all_fiducial_pdfs_.shape)

# PDFs at z
all_fiducial_pdfs_R_z = np.stack([
    all_fiducial_pdfs_[z_ix, ...] for z_ix in z_idx], axis=0)

print(all_fiducial_pdfs_R_z.shape)

# PDFs at R
all_fiducial_pdfs_R_z = np.stack([
    all_fiducial_pdfs_R_z[..., R] for R in R_idx], axis=-1)

print(all_fiducial_pdfs_R_z.shape)

In [ ]:
# Average over realisations
print(">all fids R z", all_fiducial_pdfs_R_z.shape)

fiducial_pdfs_stacked_mean = all_fiducial_pdfs_R_z.mean(axis=1)

print(">fid. stacked mean", fiducial_pdfs_stacked_mean.shape)

# Stack, for each redshift, each scale R's density bins
fiducial_pdfs_stacked_mean = np.concatenate([
    fiducial_pdfs_stacked_mean[:, :, R] for R in range(n_radii)], axis=1)

# Normalise the bins
for z, z_ix in enumerate(z_idx):
    fiducial_pdfs_stacked_mean[z] /= np.concatenate([D_deltas] * n_radii)

print(">fid. stacked mean:", fiducial_pdfs_stacked_mean.shape)

# Assuming same shape mean PDF as Cora
cdf = np.zeros((len(z_idx), n_radii * n_bins_pdf))
for z, z_ix in enumerate(z_idx):
    for i in range(1, n_bins_pdf):
        for R in range(n_radii):
            # 'z' index here is correct, on fid mean too
            # Loop over z, each R's set of bins, add probability in bin to cdf bin before
            cdf[z, R * n_bins_pdf + i] = cdf[z, R * n_bins_pdf + i - 1] + \
                fiducial_pdfs_stacked_mean[z, R * n_bins_pdf + i - 1] * D_deltas[i - 1] 

    # Should be 0. and 1.
    # print(">cdf min/max", cdf[z].min(), cdf[z].max())

print(">fid. pdfs min/max", fiducial_pdfs_stacked_mean.min(axis=1), fiducial_pdfs_stacked_mean.max(axis=1))

# Cut indices for each value of R
cuts = []
for z, z_ix in enumerate(z_idx):
    cuts_z = [
        np.where(
            (cdf[z, R * n_bins_pdf : (R + 1) * n_bins_pdf] > p_value_min) & \
            (cdf[z, R * n_bins_pdf : (R + 1) * n_bins_pdf] < p_value_max))[0] # grab the idx out of tuple
        for R in range(n_radii)]
    dims = [R.shape[0] for R in cuts_z]
    print(f"CUTS: {dims} {sum(dims)}")
    cuts.append(cuts_z)
print(sum([sum(_.shape) for _ in cuts_z]))

all_z_cut_dim_totals = 0
for z, cut in enumerate(cuts):
    print(f"\nz={z}")
    z_cut_dim_total = 0
    for R, cut_z in zip(R_values, cut):
        print(f" R={R} cut: {cut_z.shape}", end="")
        z_cut_dim_total += cut_z.shape[0]
    print(f" all: {z_cut_dim_total} / {len(R_values) * n_bins_pdf}")
    all_z_cut_dim_totals += z_cut_dim_total
print("all cuts added:", all_z_cut_dim_totals)

print(f"\n>CDF shape: {cdf.shape}")
print(f"Total bins dropped in CDF cut: {all_z_cut_dim_totals}/{len(redshifts) * len(R_values) * n_bins_pdf}")

In [ ]:
# Cuts from CDF equal CDF in probability range of cut
# print(55 + 40 + 31 + 25 + 46 + 33 + 24 + 20 + 39 + 27 + 20 + 16)
print(len(cuts), len(cuts[0]))
cuts_dim_total = 0
for z_cut in cuts:
    cut_dims = [np.prod(_.shape) for _ in z_cut]
    e = sum(cut_dims)
    print(e, cut_dims)
    cuts_dim_total += e
    
print(n_bins_pdf * n_radii * len(redshifts) - cuts_dim_total)

print(cuts_dim_total)
np.where((cdf > 0.001) & (cdf < 0.999))[0].shape

In [ ]:
for z, cut in enumerate(cuts):
    print(f"\nz={z}:")
    for R, cut_z in zip(R_values, cut):
        print(f" R={R} cut: {cut_z.shape}", end="")

In [ ]:
fig, axs = plt.subplots(
    1, len(z_idx), figsize=(3. * len(z_idx), 3.), dpi=200, sharey=True) 

pdf_dim = 99  

# plt.plot(cdf, label="cdf", color="goldenrod")
for z, z_ix in enumerate(z_idx):
    ax = axs[z] if len(z_idx) > 1 else axs
    ax.set_title(f"CDF cuts ({p_value_min}) z={redshifts[z]}")

    cut_maxima = [] # for plot ticks
    for R in range(n_radii):
        # indices in flattened datavector for R value 
        ix_1, ix_2 = R * n_bins_pdf, (R + 1) * n_bins_pdf

        ax.plot(
            range(ix_1, ix_2), 
            cdf[z, R * n_bins_pdf : (R + 1) * n_bins_pdf], 
            color="goldenrod")
    
        # the cuts for each pdf at redshift 'z'
        ix = np.where(
            (cdf[z, ix_1:ix_2] > p_value_min) & \
            (cdf[z, ix_1:ix_2] < p_value_max))[0]

        # indices for plotting high/low density CDF cut
        plot_ix = np.argwhere(
            (cdf[z, ix_1:ix_2] > p_value_min) & \
            (cdf[z, ix_1:ix_2] < p_value_max)) + (R * n_bins_pdf)

        ax.plot(
            np.arange(min(plot_ix), max(plot_ix) + 1),
            cdf[z, plot_ix], 
            label=f"cdf cut R{R}", 
            color="rebeccapurple")
        ax.vlines(x=R * n_bins_pdf, ymin=-1., ymax=2., color="k", linewidth=0.5)

        x = np.arange(min(plot_ix), max(plot_ix) + 1)
        y = cdf[z, plot_ix].squeeze() # this hsould be z no z_ix?
        a, b = min(plot_ix), max(plot_ix) + 1
        ax.fill_between(x, y, where=(x >= a) & (x <= b), color='gray', alpha=0.2)

        cut_maxima.append(len(cdf[z, plot_ix]))

    ticks = [pdf_dim * R + int(pdf_dim / 2) for R in range(n_radii)] 
    radii_strings = [f"$R_{n}$" for n in R_idx]
    ax.set_xticks(ticks, radii_strings, size="x-large")
    ax.set_ylim(-0.05, 1.05)

plt.subplots_adjust(hspace=0., wspace=0.)
# plt.legend()
plt.show()

In [ ]:
fig, axs = plt.subplots(
    1, len(z_idx), figsize=(3. * len(z_idx), 3.), dpi=200, sharey=True) 
fig.suptitle(f"CDF cuts: [{p_value_min},{p_value_max}]", y=1.02)

# z_ix indexes 'pdf' because 'pdf' includes all z

log_scale = False # cut part only occupies two orders of mag?
if not log_scale:
    plotter = lambda ax: ax.plot
else:
    plotter = lambda ax: ax.semilogy

for z, z_ix in enumerate(z_idx):
    ax = axs[z] if len(z_idx) > 1 else axs
    ax.set_title(f"z={redshifts[z]}")
    cut_maxima = [] # for plot ticks
    for R in range(n_radii):
        # indices in R value 
        ix_1, ix_2 = R * n_bins_pdf, (R + 1) * n_bins_pdf

        plotter(ax)(
            range(ix_1, ix_2), 
            # 'pdf' has all redshifts included
            pdf[z_ix, R * n_bins_pdf : (R + 1) * n_bins_pdf], 
            color="goldenrod")
        # the cut
        ix = np.where(
            (cdf[z, ix_1:ix_2] > p_value_min) & \
            (cdf[z, ix_1:ix_2] < p_value_max))[0]

        # indices for plotting high/low density CDF cut
        plot_ix = np.argwhere(
            (cdf[z, ix_1:ix_2] > p_value_min) & \
            (cdf[z, ix_1:ix_2] < p_value_max)) + (R * n_bins_pdf)

        plotter(ax)(
            np.arange(min(plot_ix), max(plot_ix) + 1),
            pdf[z_ix, plot_ix], 
            label=f"cdf cut R{R}", 
            color="rebeccapurple")
        # ax.fill_between([plot_ix.min(), plot_ix.max()], y1=pdf[z_ix, plot_ix], color="goldenrod")
        ax.axvline(x=R * n_bins_pdf, color="k", linewidth=0.5)

        x = np.arange(min(plot_ix), max(plot_ix) + 1)
        y = pdf[z_ix, plot_ix].squeeze()
        a, b = min(plot_ix), max(plot_ix) + 1
        ax.fill_between(x, y, where=(x >= a) & (x <= b), color='gray', alpha=0.2)

        cut_maxima.append(len(cdf[z, plot_ix]))

    ax.axvline(x=(R + 1) * n_bins_pdf, color="k", linewidth=0.5)
    ticks = [pdf_dim * R + int(pdf_dim / 2) for R in range(n_radii)] 
    radii_strings = [f"$R_{n}$" for n in R_idx]
    ax.set_xticks(ticks, radii_strings, size="x-large")

plt.subplots_adjust(hspace=0., wspace=0.)
# plt.legend()
plt.show()

#### cut derivatives

In [ ]:
# Excluding w, Mv
fiducial_parameters = np.array([
  0.3175, 0.049, 0.6711, 0.9624, 0.834])

parameter_strings = [
  r"$\Omega_m$", "$\Omega_b$", "$h_m$", "$n_s$", "$\sigma_8$"]

# Names of parameter derivative folders
parameter_derivative_names = [
  ["Om_m", "Om_p"], 
  ["Ob2_m", "Ob2_p"], # Larger dtheta for Ob
  ["h_m",  "h_p"], 
  ["ns_m", "ns_p"], 
  ["s8_m", "s8_p"]]

# For each param, shift in parameter value for taking derivative, 
# following ordering of 'parameter_derivative_names' and fiducial parameters 
# above 
derivative_percentages = np.array([0.018, 0.04, 0.03, 0.02, 0.018])

In [ ]:
print(redshifts)

n_derivatives = 500 
n_params = len(fiducial_parameters)

# Files use these strings for redshifts (quijote)
redshift_strings = ["0", "0.5", "1", "2", "3"]
redshift_strings_ = [redshift_strings[_] for _ in z_idx]

dparams = derivative_percentages * fiducial_parameters

# Derivatives for all z at each R, for all parameters
all_z_pm_derivatives = []
for z_ix_, redshift in enumerate(redshifts):

  derivatives = np.zeros((n_derivatives, n_params, n_radii, 2, n_bins_pdf))
  bad_idx = []
  z_string = redshift_strings_[z_ix_]

  # Each realisation's derivative
  for n_d in range(n_derivatives):
    # Each parameter
    for p, parameter_derivative in enumerate(parameter_derivative_names):
      # Each plus/minus derivative
      for pm, p_or_m in enumerate(parameter_derivative):

        # Directory of a realisation with all parameter derivatives in
        derivative_dir = os.path.join(
            derivatives_dir, p_or_m, str(n_d) + "/") 
        
        for R, R_value in enumerate(R_values):
          # Derivative of pdf w.r.t. param 
          msg = f"\rz={redshift:.1f} R={R_value} n_d={n_d:04d}"
          try:
            if resolution == 1024:
              filename_ = f"PDF_m_{resolution}_{R_value}_z={z_string}.txt"
            else: 
              filename_ = f"PDF_m_{R_value}_z={z_string}.txt"

            filename_ = os.path.join(derivative_dir, filename_)

            # correctly unpacked 
            deltas, dpdf_dparam = np.loadtxt(filename_, unpack=True)

            # n-th derivative of parameter p, pdf at scale R, +/- derivative
            derivatives[n_d, p, R, pm] = dpdf_dparam
            
            msg = msg + f"file={filename_}"

          except ValueError:
            msg += f" (BAD PDF {n_d:04d})" 
            bad_idx.append(n_d)
            pass

        print(msg, end="")
                
  # Once acquired plus/minus derivatives 
  if len(bad_idx) > 0:
    derivatives = np.delete(derivatives, bad_idx, axis=0)
            
  # (theta_p - theta_m) -> (n, p, R, d)
  derivatives = derivatives[..., 1, :] - derivatives[..., 0, :]

  # (theta_p - theta_m) / (2 * dtheta)
  pm_derivatives = np.zeros((n_derivatives - len(bad_idx), n_params, n_radii, n_bins_pdf))
  for n_d in range(n_derivatives - len(bad_idx)):
      for p in range(n_params):
        for R, R_ in enumerate(R_values):
            # derivatives: (n, p, R, d)
            numerical_derivative = derivatives[n_d, p, R, :] / (2. * dparams[p])

            pm_derivatives[n_d, p, R, :] = numerical_derivative

            print(f"\r z={redshift} R={R_} n_d={n_d:04d} bad={len(bad_idx)}", end="")

  print(f"\n derivatives & pm-derivatives: {derivatives.shape}, {pm_derivatives.shape}")

  all_z_pm_derivatives.append(pm_derivatives)